In [40]:
import telebot
from telebot import types
import pandas as pd
import numpy as np
import json
import re
import os
import pyzbar
from pyzbar.pyzbar import decode
from PIL import Image
from nalog_python import NalogRuPython
from sqlalchemy import create_engine
from sqlalchemy.orm import Session, sessionmaker
import psycopg2

In [41]:
with open('config/conf.json', 'r') as f:
    conf = json.load(f)

# Создаем экземпляр бота
bot = telebot.TeleBot(conf['token'])
#  подключение к базе
engine = create_engine(conf['engine'])

In [42]:
# обработка приветствия
def hello():
    h = ['Dzien dobry', 'Гамарджоба', 'Hello', 
         'Aloha',  'Здравствуйте','Shalom', 'Привет', 'Buenas dias', 'Guten Tag', 'Здраво']
    i = np.random.randint(0,len(h))
    return h[i]

In [43]:
# на вход принимает и обрабатывает json для добавления в таблицу
def purch(x):
    purch = pd.json_normalize(x, record_path =['items'])
    purch['check_num'] = x['requestNumber']
    purch['date'] = x['localDateTime'].replace('T', ' ')
    purch['date'] = pd.to_datetime(purch['date']).dt.date
    purch['shop'] = x['user']
    purch['price'] = purch['price']/100
    purch['sum'] = purch['sum']/100 
    purch.rename(columns={'sum': 'summ'}, inplace=True)
    purch = purch[['name', 'price', 'quantity', 'summ','check_num', 'date',	'shop']]
    purch['shop'] = purch['shop'].apply(lambda x: x.upper().replace('АКЦИОНЕРНОЕ ОБЩЕСТВО', 'АО'))
    purch['shop'] = purch['shop'].apply(lambda x: x.upper().replace('ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ', 'ООО'))
    purch.to_sql("purch", con=engine, if_exists='append', index=False)
    purch.to_sql("purch_backup", con=engine, if_exists='append', index=False)


In [44]:
# открывает файл json
def js(name):
    file_name = name
    with open(file_name, 'r', encoding='utf-8') as f:
        x = json.load(f)
    purch(x)
    os.remove(name)

In [45]:
# обработка фото
def photo(src):
    d = decode(Image.open(src))
    qr = d[0].data.decode("utf-8")
    phone = conf['phone']
    qr_code = qr
    client = NalogRuPython(qr_code, phone)# ------
    os.remove(src)

    ticket = client.get_ticket()# -----
    a = json.dumps(ticket, indent=4, ensure_ascii=False)
    x = json.loads(a)
    #пересобираю json и отправляю в функцию обработки
    jsn = {}
    jsn['localDateTime'] = x['query']['date']
    jsn['requestNumber'] = x['ticket']['document']['receipt']['requestNumber']
    jsn['user'] = x['ticket']['document']['receipt']['user']
    jsn['shop'] = x['ticket']['document']['receipt']['retailPlace']
    jsn['items'] = x['ticket']['document']['receipt']['items']
    
    purch(jsn)
    return 'ok'


In [46]:
def information(call):

    # общая сумма покупок, средняя стоимость товара, самый дорогой товар, самый дешевый
    query_sum_avg = ''' 
    select round(sum(summ),2), round(avg(price),2), max(price), min(price) 
    from purch
    '''
    summa = pd.read_sql(query_sum_avg, con = engine)


    mes = f'Всего потрачено: {summa.iloc[0][0]}\nСредняя стоимость товара: {summa.iloc[0][1]}\nсамый дорогой товар: {summa.iloc[0][2]}\nСамый дешевый товар: {summa.iloc[0][3]}'
    bot.send_message(call.message.chat.id, mes)
    

In [47]:
# Функция, обрабатывающая команду /start
@bot.message_handler(commands=['start', 'help'])
def start(m, res=False):
    if m.text == '/start':

        keyboard = types.InlineKeyboardMarkup() #клавиатура
        key_information = types.InlineKeyboardButton(text='информация', callback_data='information') #кнопка
        keyboard.add(key_information)
        bot.send_message(m.chat.id, 'Я уже кое-что умею 😃',reply_markup=keyboard)

    else:
        bot.send_message(m.chat.id, 'Мне можно прислать файл JSON из приложения налоговой\nили фото чека с QR кодом.\nЯ добавлю ваши покупки в таблицу')
        
    
#Получение сообщений от юзера
@bot.message_handler(content_types=["text"])
def handle_text(message):
    
    if message.text.lower() in ['привет', 'здравствуйте']:
        bot.send_message(message.chat.id, f'{hello()} {str(message.chat.username)}')
        
        
    else:    
        bot.send_message(message.chat.id, '...' * np.random.randint(1,5))# + message.text)


#получение документа(json)
@bot.message_handler(content_types=['document'])
def handle_docs_doc(message):
    try:
        chat_id = message.chat.id

        file_info = bot.get_file(message.document.file_id)
        downloaded_file = bot.download_file(file_info.file_path)
        src = r'load/' + file_info.file_path
        with open(src, 'wb') as new_file:
            new_file.write(downloaded_file)
            
        js(name=src)
        
        bot.reply_to(message, "Пожалуй, я сохраню это")
    except Exception as e:
        bot.reply_to(message, e)        
        

        
@bot.message_handler(content_types=['photo'])
def handle_docs_photo(message):
    
    try:
        chat_id = message.chat.id

        file_info = bot.get_file(message.photo[-1].file_id)
        downloaded_file = bot.download_file(file_info.file_path)
        src = r'load/' + file_info.file_path 
        with open(src, 'wb') as new_file:
            new_file.write(downloaded_file)
           
        bot.reply_to(message, photo(src) )
    except Exception as e:
        bot.reply_to(message, e) 


@bot.callback_query_handler(func=lambda call: True)
def callback_worker(call):
    if call.data == "information": 
        
        information(call)

    elif call.data == "no":
        bot.send_message(call.message.chat.id, 'заполните заново, нажмите /start')
    
    
# Запускаем бота
bot.polling(none_stop=True, interval=0)
 